说明：   

1、softmax的输入不需要再做非线性变换，也就是说softmax之前不再需要激活函数(relu)。softmax两个作用，如果在进行softmax前的input有负数，通过指数变换，得到正数。所有类的概率求和为1。 

2、y的标签编码方式是one-hot。对one-hot的理解是只有一位是1，其他位为0。(但是标签的one-hot编码是算法完成的，算法的输入仍为原始标签)

3、多分类问题，标签y的类型是LongTensor。比如说0-9分类问题，如果y = torch.LongTensor([3])，对应的one-hot是[0,0,0,1,0,0,0,0,0,0].(这里要注意，如果使用了one-hot，标签y的类型是LongTensor，糖尿病数据集中的target的类型是FloatTensor)

4、CrossEntropyLoss <==> LogSoftmax + NLLLoss。也就是说使用CrossEntropyLoss最后一层(线性层)是不需要做其他变化的；使用NLLLoss之前，需要对最后一层(线性层)先进行SoftMax处理，再进行log操作。 

---
代码说明：   

1、第8讲 from torch.utils.data import Dataset，第9讲 from torchvision import datasets。该datasets里面init，getitem,len魔法函数已实现。

2、torch.max的返回值有两个，第一个是每一行的最大值是多少，第二个是每一行最大值的下标(索引)是多少。

3、全连接神经网络

4、torch.no_grad() :https://blog.csdn.net/Answer3664/article/details/99460175  Python中with的用法:https://blog.csdn.net/ego_bai/article/details/80873242

5、代码中"_"的说明  Python中各种下划线的操作 :https://zhuanlan.zhihu.com/p/105783765?utm_source=com.miui.notes

6、torch.max( )的用法:https://blog.csdn.net/Z_lbj/article/details/79766690  torch.max( )使用讲解:https://blog.csdn.net/qq_40210586/article/details/103874000

In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差,图片转化成（1，28，28）
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
# root 为下载地址，traind等于True 为训练集
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
 
 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
 
    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch,  28*28=784; (1,28,28)-->(1,784),(1,28,28)为（图片通道数，宽，高）
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)  # 最后一层不做激活，不进行非线性变换，留着后面做交叉熵损失计算
 
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()#交叉熵损失计算 包括Softmax--> Log --> (-YlogY^) *(one hot)--> loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


[1,   300] loss: 2.151
[1,   600] loss: 0.710
[1,   900] loss: 0.408
accuracy on test set: 90 % 
[2,   300] loss: 0.314
[2,   600] loss: 0.262
[2,   900] loss: 0.216
accuracy on test set: 94 % 
[3,   300] loss: 0.179
[3,   600] loss: 0.166
[3,   900] loss: 0.155
accuracy on test set: 95 % 
[4,   300] loss: 0.126
[4,   600] loss: 0.124
[4,   900] loss: 0.118
accuracy on test set: 95 % 
[5,   300] loss: 0.101
[5,   600] loss: 0.095
[5,   900] loss: 0.092
accuracy on test set: 96 % 
[6,   300] loss: 0.074
[6,   600] loss: 0.076
[6,   900] loss: 0.082
accuracy on test set: 97 % 
[7,   300] loss: 0.063
[7,   600] loss: 0.060
[7,   900] loss: 0.066
accuracy on test set: 97 % 
[8,   300] loss: 0.050
[8,   600] loss: 0.049
[8,   900] loss: 0.055
accuracy on test set: 97 % 
[9,   300] loss: 0.043
[9,   600] loss: 0.041
[9,   900] loss: 0.039
accuracy on test set: 97 % 
[10,   300] loss: 0.031
[10,   600] loss: 0.036
[10,   900] loss: 0.036
accuracy on test set: 97 % 
